In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import argparse
import sys,os
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import argparse
from datetime import datetime
import hashlib
import os.path
import random
import re
import sys
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat
import pandas as pd
import cv2


import os
print(os.listdir("wheat_data/test_128"))

In [ ]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
# Define transforms for the training and validation sets
data_transforms ={ "test_transforms": transforms.Compose([#transforms.Resize(128),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
}

In [ ]:
test_root_clean = 'wheat_data/test_128/clean_imgs'
test_root_blur = 'wheat_data/test_128/blur_imgs'
test_root_deblur = 'wheat_data/test_128/deblur_imgs'
test_root_spvd_deblur = 'wheat_data/test_128/deblur_spvd_imgs'



dir_test = test_root_clean  # blur test
dir_blur = test_root_blur   # blur test
dir_deblur = test_root_deblur  # deblur test
dir_deblur_spvd = test_root_spvd_deblur  # deblur test


# Load the datasets with ImageFolder
test_data = datasets.ImageFolder(dir_test, transform=data_transforms["test_transforms"])

# Load the datasets with ImageFolder
test_data_blur = datasets.ImageFolder(dir_blur, transform=data_transforms["test_transforms"])

# Load the datasets with ImageFolder
test_data_deblur = datasets.ImageFolder(dir_deblur, transform=data_transforms["test_transforms"])

# Load the datasets with ImageFolder
test_data_deblur_spvd = datasets.ImageFolder(dir_deblur_spvd, transform=data_transforms["test_transforms"])

clean_testloader = torch.utils.data.DataLoader(test_data, batch_size = 32)
blur_testloader  = torch.utils.data.DataLoader(test_data_blur, batch_size = 32)
deblur_testloader  = torch.utils.data.DataLoader(test_data_deblur, batch_size = 32)
spvd_deblur_testloader  = torch.utils.data.DataLoader(test_data_deblur_spvd, batch_size = 32)


In [ ]:
classes=['healthy_wheat', 'leaf_rust', 'stem_rust']

In [ ]:
trained_path = 'trained_models/googlenet.pt'
# Specify model architecture
# Load the pretrained model from pytorch's library and stored it in model_transfer
model_transfer = models.googlenet(pretrained=True)

# Check if GPU is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()
    
#print the model to see all the layers
#print(model_transfer)

for param in model_transfer.parameters():
    param.requires_grad=True
    
# Define n_inputs takes the same number of inputs from pre-trained model
n_inputs = model_transfer.fc.in_features #refer to the fully connected layer only

# Add last linear layer (n_inputs -> 4 classes). In this case the ouput is 4 classes
# New layer automatically has requires_grad = True
last_layer = nn.Linear(n_inputs, len(classes))

model_transfer.fc = last_layer

# If GPU is available, move the model to GPU
if use_cuda:
    model_transfer = model_transfer.cuda()
  
# Check to see the last layer produces the expected number of outputs
print(model_transfer.fc.out_features)

# Load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load(trained_path))

In [ ]:
criterion_transfer = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()


In [ ]:
predicted = []
actual = []
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model_transfer.eval() #set model into evaluation/testing mode. It turns of drop off layer
    #Iterating over test data
    for batch_idx, (data, target) in enumerate(loaders):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        actual.append(np.squeeze(target.cpu().numpy()))
        predicted.append(np.squeeze(pred.cpu().numpy()))
        
        # compare predictions to 
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [ ]:
classes=['healthy', 'leaf_rust', 'stem_rust']

In [ ]:
 def plot_confusion_matrix(cm, title='Confusion matrix',figsize=None, cmap=plt.cm.Reds, labels=[]):
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    plt.title(title,fontsize=14)
    plt.colorbar()
    labels = classes
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, fontsize=12,rotation=45)
    plt.yticks(tick_marks, labels, fontsize=12 ,rotation=0)
    plt.tight_layout()
    plt.xlabel('Predicted label',fontsize=13)
    plt.ylabel('True label',fontsize=13)
#    plt.savefig('confusionmatrix_figs/'+title+'.png')

#### Clean Images  Accuracy

In [ ]:
testloader = clean_testloader

predicted = []
actual = []
# call test function    
test(testloader, model_transfer, criterion_transfer, use_cuda)

actual_label=[]
for arr in actual:
    for l in arr:
        actual_label.append(l)
actual_label
target_label=[]
for arr in predicted:
    for l in arr:
        target_label.append(l)
        
y_true = actual_label
y_pred = target_label
target_names = classes

cm = confusion_matrix(y_true, y_pred)
print('Final Test Accuracy', accuracy_score(y_true, y_pred))
plot_confusion_matrix(cm, title = 'Clean Images',figsize=(4.5,3.5))
# print(classification_report(y_true, y_pred, target_names=target_names))
# print('Final Test Accuracy', accuracy_score(y_true, y_pred))

#### Blur Images  Accuracy

In [ ]:
testloader = blur_testloader

predicted = []
actual = []
# call test function    
test(testloader, model_transfer, criterion_transfer, use_cuda)

actual_label=[]
for arr in actual:
    for l in arr:
        actual_label.append(l)
actual_label
target_label=[]
for arr in predicted:
    for l in arr:
        target_label.append(l)

y_true = actual_label
y_pred = target_label
target_names = classes

cm = confusion_matrix(y_true, y_pred)
print('Final Test Accuracy', accuracy_score(y_true, y_pred))
plot_confusion_matrix(cm, title = 'Blur Images',figsize=(4.5,3.5))

#### Unsupervised DeBlur Images  Accuracy

In [1]:
testloader = deblur_testloader

predicted = []
actual = []
# call test function    
test(testloader, model_transfer, criterion_transfer, use_cuda)

actual_label=[]
for arr in actual:
    for l in arr:
        actual_label.append(l)
actual_label
target_label=[]
for arr in predicted:
    for l in arr:
        target_label.append(l)
        
        
y_true = actual_label
y_pred = target_label
target_names = classes

cm = confusion_matrix(y_true, y_pred)
print('Final Test Accuracy', accuracy_score(y_true, y_pred))

plot_confusion_matrix(cm, title = 'Unsupervised',figsize=(4.5,3.5))

NameError: name 'deblur_testloader' is not defined

#### supervised DeBlur Images  Accuracy

In [ ]:
testloader = spvd_deblur_testloader

predicted = []
actual = []
# call test function    
test(testloader, model_transfer, criterion_transfer, use_cuda)

actual_label=[]
for arr in actual:
    for l in arr:
        actual_label.append(l)
actual_label
target_label=[]
for arr in predicted:
    for l in arr:
        target_label.append(l)
        
        
y_true = actual_label
y_pred = target_label
target_names = classes

cm = confusion_matrix(y_true, y_pred)

plot_confusion_matrix(cm, title = 'Supervised',figsize=(4.5,3.5))

print(classification_report(y_true, y_pred, target_names=target_names))
print('Final Test Accuracy', accuracy_score(y_true, y_pred))